## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Kapaa,US,22.0752,-159.3190,84.18,78,90,4.00,overcast clouds,2021-08-01 21:37:21
1,1,Qui Nhon,VN,13.7667,109.2333,84.47,68,91,12.48,overcast clouds,2021-08-01 21:37:21
2,2,Formosa,AR,-26.1775,-58.1781,69.78,43,0,3.44,clear sky,2021-08-01 21:33:18
3,3,Severo-Kurilsk,RU,50.6789,156.1250,55.53,73,0,14.61,clear sky,2021-08-01 21:37:22
4,4,Khasan,RU,42.4308,130.6434,75.56,91,95,19.37,overcast clouds,2021-08-01 21:37:22


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Kapaa,US,22.0752,-159.3190,84.18,78,90,4.00,overcast clouds,2021-08-01 21:37:21
1,1,Qui Nhon,VN,13.7667,109.2333,84.47,68,91,12.48,overcast clouds,2021-08-01 21:37:21
4,4,Khasan,RU,42.4308,130.6434,75.56,91,95,19.37,overcast clouds,2021-08-01 21:37:22
17,17,Puerto Ayora,EC,-0.7393,-90.3518,78.76,87,51,7.00,broken clouds,2021-08-01 21:37:26
20,20,Albany,US,42.6001,-73.9662,75.51,72,86,7.52,overcast clouds,2021-08-01 21:36:50
21,21,Cattolica,IT,43.9575,12.7376,80.47,40,96,10.00,overcast clouds,2021-08-01 21:37:27
23,23,Hilo,US,19.7297,-155.0900,80.33,69,90,4.61,light rain,2021-08-01 21:37:28
24,24,Tual,ID,-5.6667,132.7500,81.27,78,53,12.17,broken clouds,2021-08-01 21:37:28
27,27,Aklavik,CA,68.2191,-135.0107,83.23,36,98,18.03,overcast clouds,2021-08-01 21:37:29
28,28,Guerrero Negro,MX,27.9769,-114.0611,83.82,54,3,13.38,clear sky,2021-08-01 21:37:29


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
Date                   197
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,
1,Qui Nhon,VN,84.47,overcast clouds,13.7667,109.2333,
4,Khasan,RU,75.56,overcast clouds,42.4308,130.6434,
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
20,Albany,US,75.51,overcast clouds,42.6001,-73.9662,
21,Cattolica,IT,80.47,overcast clouds,43.9575,12.7376,
23,Hilo,US,80.33,light rain,19.7297,-155.0900,
24,Tual,ID,81.27,broken clouds,-5.6667,132.7500,
27,Aklavik,CA,83.23,overcast clouds,68.2191,-135.0107,
28,Guerrero Negro,MX,83.82,clear sky,27.9769,-114.0611,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
1,Qui Nhon,VN,84.47,overcast clouds,13.7667,109.2333,Hoang Yen Canary Hotel
4,Khasan,RU,75.56,overcast clouds,42.4308,130.6434,
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Albany,US,75.51,overcast clouds,42.6001,-73.9662,
21,Cattolica,IT,80.47,overcast clouds,43.9575,12.7376,Hotel Antares
23,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
24,Tual,ID,81.27,broken clouds,-5.6667,132.7500,Grand Vilia Hotel
27,Aklavik,CA,83.23,overcast clouds,68.2191,-135.0107,Aklavik Inn
28,Guerrero Negro,MX,83.82,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"


In [21]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
# replace field that's entirely space (or empty) with NaN
clean_hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
1,Qui Nhon,VN,84.47,overcast clouds,13.7667,109.2333,Hoang Yen Canary Hotel
4,Khasan,RU,75.56,overcast clouds,42.4308,130.6434,NaN
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Albany,US,75.51,overcast clouds,42.6001,-73.9662,NaN
21,Cattolica,IT,80.47,overcast clouds,43.9575,12.7376,Hotel Antares
23,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
24,Tual,ID,81.27,broken clouds,-5.6667,132.7500,Grand Vilia Hotel
27,Aklavik,CA,83.23,overcast clouds,68.2191,-135.0107,Aklavik Inn
28,Guerrero Negro,MX,83.82,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"


In [22]:
# 7. Drop the rows where there is no Hotel Name.
new_hotel_df = clean_hotel_df.dropna(axis=0,how='any')
new_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
1,Qui Nhon,VN,84.47,overcast clouds,13.7667,109.2333,Hoang Yen Canary Hotel
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Cattolica,IT,80.47,overcast clouds,43.9575,12.7376,Hotel Antares
23,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
24,Tual,ID,81.27,broken clouds,-5.6667,132.7500,Grand Vilia Hotel
27,Aklavik,CA,83.23,overcast clouds,68.2191,-135.0107,Aklavik Inn
28,Guerrero Negro,MX,83.82,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
33,Dingle,PH,77.95,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
38,Cabra,ES,78.24,broken clouds,37.4725,-4.4421,MS Fuente Las Piedras


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))